In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
def create_dataset(X_place, y_place, batch_size, num_epochs):
    dataset = tf.data.Dataset.from_tensor_slices((X_place, y_place))
    # X_place [num_samples, num_features]
    dataset = dataset.shuffle(5000)
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)
    return dataset

In [3]:
def create_model(iterator, mode):
    x, y = iterator.get_next()
    
    hidden = tf.layers.dense(x, 400, activation=tf.nn.sigmoid)
    hidden = tf.layers.dense(hidden, 100, activation=tf.nn.sigmoid)
    
    out = tf.layers.dense(hidden, 1, activation=tf.nn.sigmoid)
    
    loss = tf.losses.mean_squared_error(y, out)
    tf.summary.scalar("MSE", loss, collections=["summs"])
    
    opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    # opt = tf.train.MomentumOptimizer(...)
    update = opt.minimize(loss)
    
    return loss, out, update

In [4]:
data = pd.read_csv("./normalized.csv")
X_data = data.iloc[:, 0:-1]
y_data = data.iloc[:, [-1]]

In [5]:
y_data.shape

(2329, 1)

In [6]:
X_placeholder = tf.placeholder(tf.float32, shape=X_data.shape)
y_placeholder = tf.placeholder(tf.float32, shape=y_data.shape)
dataset = create_dataset(X_placeholder, y_placeholder, 5, 1)
iterator = dataset.make_initializable_iterator()
loss, out, update = create_model(iterator, "TRAIN")

In [7]:
writer = tf.summary.FileWriter("./housing/")
writer.add_graph(tf.get_default_graph())

In [8]:
summaries = tf.summary.merge_all("summs")

In [9]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.run(iterator.initializer,
             feed_dict={X_placeholder: X_data,
                        y_placeholder: y_data})
    step = 0
    while True:
        try:
            step += 1
            tr_loss, _, summ = sess.run([loss, update, summaries])
            if step % 50 == 0:
                print("Step %d: %3.3f" % (step, tr_loss))
                writer.add_summary(summ, step)
        except tf.errors.OutOfRangeError:
            print("Training done.")
            writer.close()
            break

Step 50: 0.020
Step 100: 0.004
Step 150: 0.012
Step 200: 0.013
Step 250: 0.008
Step 300: 0.034
Step 350: 0.056
Step 400: 0.005
Step 450: 0.011
Training done.
